- Buy when an upward Renko brick forms after a downward brick or if its the first brick and no position is currently held.
- Sell when a downward Renko brick forms after an upward brick if position is currently held.

In [8]:
import pandas as pd
import yfinance as yf

In [9]:
df=yf.download('AAPL',start='2020-01-01')

[*********************100%%**********************]  1 of 1 completed


In [10]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.960464,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.251152,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.826859,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.484337,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.650345,132079200
...,...,...,...,...,...,...
2024-07-25,218.929993,220.850006,214.619995,217.490005,217.490005,51391200
2024-07-26,218.699997,219.490005,216.009995,217.960007,217.960007,41601300
2024-07-29,216.960007,219.300003,215.750000,218.240005,218.240005,36311800


In [11]:
def create_renko(df,brick_size):
    renko_list=[]
    previous_price=df.iloc[0]['Adj Close']
    for index,row in df.iterrows():
        price=row['Adj Close']
        while abs(price - previous_price) >= brick_size:
            if price > previous_price:
                previous_price+=brick_size
                renko_list.append({'Date': index,'Close': previous_price,'Brick': 'up'})
            else:
                previous_price-= brick_size
                renko_list.append({'Date':index,'Close':previous_price,'Brick':'down'})
    renko_df=pd.DataFrame(renko_list)
    return renko_df

In [12]:
renko_df=create_renko(df,20)

In [13]:
renko_df

,Date,Close,Brick
0,2020-07-08,92.960464,up
1,2020-08-18,112.960464,up
2,2020-12-28,132.960464,up
3,2021-09-07,152.960464,up
4,2021-12-10,172.960464,up
5,2022-03-11,152.960464,down
6,2022-03-28,172.960464,up
7,2022-05-09,152.960464,down
8,2022-06-13,132.960464,down
9,2022-07-21,152.960464,up


In [14]:
initial_balance=10000
balance=initial_balance
position=0

for i in range(len(renko_df)):
    #Buy signal : Upward reversal or the first upward brick
    if renko_df.iloc[i]['Brick']== 'up' and (i==0 or renko_df.iloc[i-1]['Brick']=='down'):
        if position == 0: # Ensure we are not already holding a position
            buy_price=renko_df.iloc[i]['Close']
            position= balance/buy_price
            balance=0
            print(f"Bought at {buy_price} on {renko_df.iloc[i]['Date']}")
    #Sell signal: downward reversal    
    elif renko_df.iloc[i]['Brick']== 'down' and renko_df.iloc[i-1]['Brick']== 'up' :
        
        if position >0:
            sell_price =renko_df.iloc[i]['Close']
            balance= position * sell_price
            position =0  #reset postion
            print(f"Sold at {sell_price} on {renko_df.iloc[i]['Date']}")
if position >0 :
    balance = position*renko_df.iloc[-1]['Close']

print(f"Initial balance : {initial_balance}")
print(f"Final balance: {balance} ")

Bought at 92.96046447753906 on 2020-07-08 00:00:00
Sold at 152.96046447753906 on 2022-03-11 00:00:00
Bought at 172.96046447753906 on 2022-03-28 00:00:00
Sold at 152.96046447753906 on 2022-05-09 00:00:00
Bought at 152.96046447753906 on 2022-07-21 00:00:00
Sold at 132.96046447753906 on 2022-12-19 00:00:00
Bought at 152.96046447753906 on 2023-02-03 00:00:00
Sold at 172.96046447753906 on 2023-09-26 00:00:00
Bought at 192.96046447753906 on 2023-12-07 00:00:00
Sold at 172.96046447753906 on 2024-03-05 00:00:00
Bought at 192.96046447753906 on 2024-06-03 00:00:00
Initial balance : 10000
Final balance: 15478.06382897406 


In [15]:
renko_df # sold at down signal

,Date,Close,Brick
0,2020-07-08,92.960464,up
1,2020-08-18,112.960464,up
2,2020-12-28,132.960464,up
3,2021-09-07,152.960464,up
4,2021-12-10,172.960464,up
5,2022-03-11,152.960464,down
6,2022-03-28,172.960464,up
7,2022-05-09,152.960464,down
8,2022-06-13,132.960464,down
9,2022-07-21,152.960464,up


In [16]:
#https://www.youtube.com/watch?v=f10jVkUS39w